In [ ]:
import sys
import gc
import torch
import torch.nn as nn
import yaml
from ultralytics import YOLO
from ultralytics.nn import tasks, modules

# ======================================================================================
# 1. CBAM 클래스 정의 및 시스템 등록
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM_Universal(nn.Module):
    def __init__(self, *args, **kwargs): 
        super(CBAM_Universal, self).__init__()
        c1 = args[0]
        k = 7 
        # 로그 출력 (디버깅용)
        print(f"✅ CBAM 생성됨 | 적용 채널: {c1}")
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)

    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# 시스템 등록
target_name = 'CBAM_Universal'
for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)
if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)
print(f"✅ [준비 완료] 모듈 '{target_name}' 시스템 등록됨.")


# ======================================================================================
# 2. YAML 파일 생성 (숫자 최종 수정: 192, 384, 576)
# ======================================================================================
cbam_yaml = {
    'nc': 3, 
    'scales': { 'm': [0.67, 0.75, 768] },
    'backbone': [
        [-1, 1, 'Conv', [64, 3, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C2f', [128, True]],
        [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C2f', [256, True]],
        # ▼ [수정] 144 -> 192 (오류 로그 got 192 반영)
        [-1, 1, target_name, [192]], # 5
        [-1, 1, 'Conv', [512, 3, 2]], [-1, 6, 'C2f', [512, True]],
        # ▼ [수정] 288 -> 384
        [-1, 1, target_name, [384]], # 8
        [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C2f', [1024, True]],
        # ▼ [수정] 576 (이건 맞음)
        [-1, 1, target_name, [576]], # 11
        [-1, 1, 'SPPF', [1024, 5]]   # 12
    ],
    'head': [
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 8], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 5], 1, 'Concat', [1]], [-1, 3, 'C2f', [256]],
        [-1, 1, 'Conv', [256, 3, 2]], [[-1, 15], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'Conv', [512, 3, 2]], [[-1, 11], 1, 'Concat', [1]], [-1, 3, 'C2f', [1024]],
        [[18, 21, 24], 1, 'Detect', ['nc']]
    ]
}
yaml_name = 'yolov8m_cbam_real_final.yaml'
with open(yaml_name, 'w') as f:
    yaml.dump(cbam_yaml, f, sort_keys=False)
print(f"✅ [준비 완료] YAML 파일({yaml_name}) 생성됨.")


# ======================================================================================
# 3. 순차적 학습 시작 (640 -> 800 -> 1280)
# ======================================================================================
data_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"

# --- [Step 1] 640 사이즈 (Batch 16) ---
print("\n🚀 [1/3] 640 사이즈 학습 시작 (Batch 16, Epochs 200)")
# 주의: 이전에 학습된게 있다면 지우거나 이름을 바꿔야 충돌이 안날 수 있음. 여기선 덮어쓰기 허용.
model_640 = YOLO(yaml_name).load('yolov8m.pt')
model_640.train(
    data=data_path, epochs=200, patience=50,
    imgsz=640, batch=16,
    optimizer='AdamW', lr0=0.001, lrf=0.01, cos_lr=True,
    mosaic=1.0, mixup=0.1, close_mosaic=10,
    device=0, workers=4,
    name='tank_yolo_cbam_640_final', exist_ok=True
)
del model_640
gc.collect()
torch.cuda.empty_cache()


# --- [Step 2] 800 사이즈 (Batch 8) ---
print("\n🚀 [2/3] 800 사이즈 학습 시작 (Batch 8, Epochs 150)")
model_800 = YOLO(yaml_name).load('yolov8m.pt')
model_800.train(
    data=data_path, epochs=150, patience=30,
    imgsz=800, batch=8, 
    optimizer='AdamW', lr0=0.001, lrf=0.01, cos_lr=True,
    mosaic=1.0, mixup=0.1, close_mosaic=10,
    device=0, workers=4,
    name='tank_yolo_cbam_800_final', exist_ok=True
)
del model_800
gc.collect()
torch.cuda.empty_cache()


# --- [Step 3] 1280 사이즈 (Batch 4) ---
print("\n🚀 [3/3] 1280 사이즈 학습 시작 (Batch 4, Epochs 150)")
model_1280 = YOLO(yaml_name).load('yolov8m.pt')
model_1280.train(
    data=data_path, epochs=150, patience=30,
    imgsz=1280, batch=4, 
    optimizer='AdamW', lr0=0.001, lrf=0.01, cos_lr=True,
    mosaic=1.0, mixup=0.1, close_mosaic=10,
    device=0, workers=4,
    name='tank_yolo_cbam_1280_final', exist_ok=True
)

print("\n🎉 모든 학습(640, 800, 1280)이 성공적으로 완료되었습니다!")

In [ ]:
import sys
import gc
import torch
import torch.nn as nn
import yaml
from ultralytics import YOLO
from ultralytics.nn import tasks, modules

# ======================================================================================
# 1. CBAM 클래스 재정의 (커널 재시작 시 필수)
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM_Universal(nn.Module):
    def __init__(self, *args, **kwargs): 
        super(CBAM_Universal, self).__init__()
        c1 = args[0]
        k = 7 
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)
    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# 시스템 등록
target_name = 'CBAM_Universal'
for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)
if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)
print(f"✅ [재등록 완료] 모듈 '{target_name}' 시스템 준비 끝.")


# ======================================================================================
# 2. YAML 파일 확인 (재생성)
# ======================================================================================
cbam_yaml = {
    'nc': 3, 
    'scales': { 'm': [0.67, 0.75, 768] },
    'backbone': [
        [-1, 1, 'Conv', [64, 3, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C2f', [128, True]],
        [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C2f', [256, True]],
        [-1, 1, target_name, [192]], # 5 (정답 채널)
        [-1, 1, 'Conv', [512, 3, 2]], [-1, 6, 'C2f', [512, True]],
        [-1, 1, target_name, [384]], # 8 (정답 채널)
        [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C2f', [1024, True]],
        [-1, 1, target_name, [576]], # 11 (정답 채널)
        [-1, 1, 'SPPF', [1024, 5]]   # 12
    ],
    'head': [
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 8], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 5], 1, 'Concat', [1]], [-1, 3, 'C2f', [256]],
        [-1, 1, 'Conv', [256, 3, 2]], [[-1, 15], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'Conv', [512, 3, 2]], [[-1, 11], 1, 'Concat', [1]], [-1, 3, 'C2f', [1024]],
        [[18, 21, 24], 1, 'Detect', ['nc']]
    ]
}
yaml_name = 'yolov8m_cbam_real_final.yaml'
with open(yaml_name, 'w') as f:
    yaml.dump(cbam_yaml, f, sort_keys=False)
print(f"✅ [준비 완료] YAML 파일 확인됨.")


# ======================================================================================
# 3. 학습 재개 (800 -> 1280) *안전 모드*
# ======================================================================================
data_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"

# --- [Step 2] 800 사이즈 (Batch 8, Cache OFF) ---
print("\n🚀 [2/3] 800 사이즈 학습 재시작 (Batch 8, Cache OFF)")
model_800 = YOLO(yaml_name).load('yolov8m.pt')
model_800.train(
    data=data_path, 
    epochs=200, 
    patience=30,
    imgsz=800, 
    batch=8, 
    
    # [중요 변경] 램 폭발 방지
    cache=False,    # 16GB RAM에서는 800 사이즈 캐싱 위험 -> 끔
    workers=4,      # 프로세스 개수 줄여서 메모리 확보
    
    optimizer='AdamW', lr0=0.001, lrf=0.01, cos_lr=True,
    mosaic=1.0, mixup=0.1, close_mosaic=10,
    device=0, 
    name='tank_yolo_cbam_800_retry', # 이름 변경 (충돌 방지)
    exist_ok=True
)
del model_800
gc.collect()
torch.cuda.empty_cache()


# --- [Step 3] 1280 사이즈 (Batch 4, Cache OFF) ---
print("\n🚀 [3/3] 1280 사이즈 학습 시작 (Batch 4, Cache OFF)")
model_1280 = YOLO(yaml_name).load('yolov8m.pt')
model_1280.train(
    data=data_path, 
    epochs=200, 
    patience=30,
    imgsz=1280, 
    batch=4, 
    
    # [중요 변경] 램 폭발 방지
    cache=False,    # 1280은 이미지 하나가 매우 커서 캐싱 절대 불가
    workers=4,
    
    optimizer='AdamW', lr0=0.001, lrf=0.01, cos_lr=True,
    mosaic=1.0, mixup=0.1, close_mosaic=10,
    device=0, 
    name='tank_yolo_cbam_1280_final', 
    exist_ok=True
)

print("\n🎉 모든 학습 스케줄이 완료되었습니다!")

      7/150      4.97G      1.828      2.198      2.285         36        800: 72% ━━━━━━━━╸─── 124/173 3.6it/s 35.3s<13.4s